
Feedback to shins777@gmail.com

이 colab은 Gemini pro 의 Multimodality의 기능으로 비디오를 분석하기 위한 예제입니다.

### 참고: 이곳에서 사용되는 비디오는 교육목적으로 웹상에서 다운로드 받은 이미지입니다. 상업적인 용도로 사용하지 말아주세요.
모든 비디오는 일부 비디오가 무료로 제공되는 https://pixabay.com 에서 다운받았습니다. 만일 저작권 문제가 있다면 해당 비디오를 삭제하도록 하겠습니다.

# Library 설치

In [7]:
!pip install --upgrade --quiet google-cloud-aiplatform

# GCP 사용자 인증

In [8]:
from google.colab import auth
auth.authenticate_user()

In [9]:
model_name="gemini-pro-vision"
project="ai-hangsik"
location="asia-northeast3"

In [10]:
from google.cloud import aiplatform
aiplatform.init(project=project)

# Gemini Pro vision - Multimodal

In [11]:
import base64
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Part


def get_obj_content(coding='utf-8', obj_loc=""):
    with open(obj_loc, 'rb') as f:
        obj_data = base64.b64encode(f.read()).decode(coding)
        return obj_data


# Video 분석

아래 예제에서 사용된 비디오는 아래 무료사이트에서 제공하는 비디오입니다. 교육의 목적으로 제공하고 있으니 상업적으로 사용하지 마시기 바랍니다.  
https://pixabay.com/videos/work-office-people-working-87593/

###video 분석 1

In [12]:
def generate_video(obj):
  model = GenerativeModel("gemini-pro-vision")
  responses = model.generate_content(
    [obj, """비디오의 내용을 전체적으로 요약해주고 아래 내용을 구체적으로 설명해주세요.
    1. 사람들이 있는 장소를 알려주고, 무엇이 있나요?
    2. 노란색옷을 입은 사람은 주로 어떤 손을 사용하나요 ?
    3. 노트북은 어떤 브랜드인가요 ?
    4. 화면에 나타난 여자의 행동은 어떤까요 ?
    4. 검정옷을 입은 사람은 손에 무엇을 들고 있나요 ?
    """],
    generation_config={
        "max_output_tokens": 2048,
        "temperature": 0.1,
        "top_p": 1,
        "top_k": 32
    },
    safety_settings=[],
    stream=True,
    )

  for response in responses:
      print(response.text, end="")

video_data = get_obj_content(obj_loc='./work(720p).mp4')
video1 = Part.from_data(data=base64.b64decode(video_data), mime_type="video/mp4")

generate_video(video1)

 1. 장소는 사무실로 보이며, 3명이 앉아 있습니다.
2. 노란색 옷을 입은 사람은 오른손을 주로 사용합니다.
3. 노트북은 애플 브랜드입니다.
4. 화면에 나타난 여자는 서류를 들고 있습니다.
5. 검정 옷을 입은 사람은 펜을 들고 있습니다.

###  video 분석 2

아래 예제에서 사용된 비디오는 아래 무료사이트에서 제공하는 비디오입니다. 교육의 목적으로 제공하고 있으니 상업적으로 사용하지 마시기 바랍니다.  
https://pixabay.com/videos/animate-teacher-class-student-22182/

In [13]:
def generate_video(obj):
  model = GenerativeModel("gemini-pro-vision")
  responses = model.generate_content(
    [obj, """전체적으로 비디오에 나타나는 상황을 자세히 설명해주세요.
    1. 주로 무엇을 그리고 있나요 ? 그리는 도구는 무엇인가요 ?
    2. 선생님의 머리길이는 어떠한가요 ?
    3. 학생은 몇명이고, 어느쪽 학생을 먼저 그리나요 ?
    4. 오른쪽의 학생은 무엇을 하고 있나요 ?
    5. 교실안에 지구본이 있나요? 어디위에 있나요 ?

    """],
    generation_config={
        "max_output_tokens": 2048,
        "temperature": 0.4,
        "top_p": 1,
        "top_k": 32
    },
    safety_settings=[],
    stream=True,
    )

  for response in responses:
      print(response.text, end="")


video_data = get_obj_content(obj_loc='./classroom(720p).mp4')
video1 = Part.from_data(data=base64.b64decode(video_data), mime_type="video/mp4")

generate_video(video1)

 1. 초등학교 교실을 그리는 것 같습니다. 도구는 흰 분필입니다.
2. 선생님의 머리길이는 짧습니다.
3. 학생은 3명이고, 오른쪽 학생을 먼저 그립니다.
4. 오른쪽 학생은 책을 읽고 있습니다.
5. 교실안에 지구본이 있습니다. 선생님의 오른쪽, 탁자 위에 있습니다.

###  video 분석 3

아래 예제에서 사용된 비디오는 아래 무료사이트에서 제공하는 비디오입니다. 교육의 목적으로 제공하고 있으니 상업적으로 사용하지 마시기 바랍니다.  


In [14]:
def generate_video(obj):
  model = GenerativeModel("gemini-pro-vision")


  responses = model.generate_content(
    [obj, """전체적으로 비디오에 나타나는 상황을 자세히 설명해주세요.
    1. 칠판에 적힌 글씨는 그대로 적어주세요.
    2. 어떤 내용의 정보를 적고 있나요 ?

    """],

    generation_config={
        "max_output_tokens": 2048,
        "temperature": 0.4,
        "top_p": 1,
        "top_k": 32
    },
    safety_settings=[],
    stream=True,
    )

  for response in responses:
      print(response.text, end="")


video_data = get_obj_content(obj_loc='./text(1080p).mp4')
video1 = Part.from_data(data=base64.b64decode(video_data), mime_type="video/mp4")

generate_video(video1)

 1. 칠판에 적힌 글씨는 다음과 같습니다.

브랜드 전략
내부 브랜드
외부 브랜드

목적: Bring everyday
비전: 
미션: 
핵심가치: 
브랜드 성격: 
브랜드 가치: 

2. 브랜드 전략에 대한 정보를 적고 있습니다.